In [314]:
import cv2

In [315]:
image = cv2.imread("assets/random.png")
image = cv2.resize(image, (720, 640))

MODEL_MEAN_VALUES = [104, 117, 123]

In [316]:
facepb_text = "models/opencv_face_detector.pbtxt"
facepb = "models/opencv_face_detector_uint8.pb"

age_prototxt = "models/age_deploy.prototxt"
age_model = "models/age_net.caffemodel"

gender_prototxt = "models/gender_deploy.prototxt"
gender_model = "models/gender_net.caffemodel"

In [317]:
# Load Models

face = cv2.dnn.readNet(facepb, facepb_text)
age = cv2.dnn.readNet(age_model, age_prototxt)
gender = cv2.dnn.readNet(gender_model, gender_prototxt)

In [318]:
age_class = ["0-2", "4-6", "8-12", "15-20", "25-32", "38-43", "48-53", "60-100"]
gender_class = ["male", "female"]

In [319]:
#copy image 
img_cp = image.copy()

In [320]:

img_h = img_cp.shape[0]
img_w = img_cp.shape[1]

blob = cv2.dnn.blobFromImage(img_cp, 1.0, (300, 300), MODEL_MEAN_VALUES, True, False)

In [321]:
face.setInput(blob)

_detected = face.forward()

In [322]:
#Draw Rectangle on faces
face_bounds = []
for i in range(_detected.shape[2]):
    confidence = _detected[0, 0, i, 2]
    if confidence >= 0.9:
        x1 = int(_detected[0, 0, i, 3] * img_w)
        y1 = int(_detected[0, 0, i, 4] * img_h) 
        x2 = int(_detected[0, 0, i, 5] * img_w)
        y2 = int(_detected[0, 0, i, 6] * img_h)
        cv2.rectangle(img_cp, (x1, y1), (x2, y2), (0, 255, 0), int(round(img_h/150)), 8)
        face_bounds.append([x1, y1, x2, y2])

In [323]:
for face_bound in face_bounds:
    try:
        face_temp = img_cp[max(0, face_bound[1]-15):
                      min(face_bound[3] + 15, img_cp.shape[0]-1), 
               max(0, face_bound[0]-15):
               min(face_bound[2] + 15, img_cp.shape[0]-1)]
        blob = cv2.dnn.blobFromImage(face_temp, 1.0, (227, 227), MODEL_MEAN_VALUES, True)
        
        #Gender
        # gender.setInput(blob)
        # _detected_gender = gender.forward()
        # gen = gender_class[_detected_gender[0].argmax()]
        
        #age
        age.setInput(blob)
        _detected_age = age.forward()
        age_f = age_class[_detected_age[0].argmax()]
        
        print(age_f)
        
    except Exception as e:
        print(e)

38-43


In [324]:
cv2.imshow("result", img_cp)
cv2.waitKey(0)
cv2.destroyAllWindows()